In [ ]:
%load_ext lab_black
%load_ext nb_black

In [ ]:
""" 
Trying black package & pandas 1.0 & large csv handling

"""

In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
size = 10000
cities = ["paris", "barcelona", "berlin", "new york"]
df = pd.DataFrame(
    {"city": np.random.choice(cities, size=size), "booked_perc": np.random.rand(size)}
)
df["id"] = df.index.map(str) + "-" + df.city
df = df[["id", "city", "booked_perc"]]
df.head()

In [ ]:
ts0 = time.time()

sum(booked_perc for booked_perc in df.booked_perc)

ts1 = time.time()
print(ts1 - ts0)

ts0 = time.time()

df.booked_perc.sum()

ts1 = time.time()
print(ts1 - ts0)

ts0 = time.time()

df["perc_change"] = df.booked_perc.pct_change()

ts1 = time.time()
print(ts1 - ts0)

df.head()

In [ ]:
## NO!
ts0 = time.time()
for i, row in df.iterrows():
    if row.city == "new york":
        df.loc[i, "sales_factor"] = row.booked_perc * 2
    else:
        df.loc[i, "sales_factor"] = 0
ts1 = time.time()
print(ts1 - ts0)

## YES
ts0 = time.time()


def calculate_sales_factor(row):
    if row.city == "new york":
        return row.booked_perc * 2
    return 0


df["sales_factor"] = df.apply(calculate_sales_factor, axis=1)
ts1 = time.time()
print(ts1 - ts0)

## YES!
ts0 = time.time()
df.loc[df.city == "new york", "sales_factor"] = (
    df[df.city == "new york"].booked_perc * 2
)
df.sales_factor.fillna(0, inplace=True)
ts1 = time.time()
print(ts1 - ts0)

df.head()

When we are dealing with function with 3 or more variables, we can break it down to multiple pandas expressions. It will be faster than using the apply function.

#### NO
f(x, a, b) = (a + b) * x

#### YES
df['a_plus_b'] = df['a'] + df['b']

df['f'] = df['a_plus_b'] * df['x']

In [ ]:
ts0 = time.time()

df.groupby("city").sales_factor.mean()
df.groupby("city").sales_factor.count()
df.groupby("city").id.first()

ts1 = time.time()
print(ts1 - ts0)

ts0 = time.time()

df.groupby("city").agg({"sales_factor": ["mean", "count"], "id": "first"})

ts1 = time.time()
print(ts1 - ts0)